## Install libs
%%capture unterdrückt die Ausgabe

In [ ]:
%%capture
!pip install --user "ibm-watsonx-ai==0.2.6"
!pip install --user "langchain==0.1.16" 
!pip install --user "langchain-ibm==0.1.4"
!pip install --user "huggingface == 0.0.1"
!pip install --user "huggingface-hub == 0.23.4"
!pip install --user "sentence-transformers == 2.5.1"
!pip install --user "chromadb"
!pip install --user "wget == 3.2"

Import der benötigen Libs

In [ ]:
# You can use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes, DecodingMethods
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
import wget

## Indexing: Load the document

In [ ]:
filename = 'companyPolicies.txt'
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/6JDbUb_L3egv_eOkouY71A.txt'

# Use wget to download the file
wget.download(url, out=filename)
print('file downloaded')

with open(filename, 'r') as file:
    # Read the contents of the file
    contents = file.read()
    print(contents)

## Indexing: Split the document into chunks

In [ ]:
loader = TextLoader(filename)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
print(len(texts))

## Indexing: Embedding and storing with [ChromaDb](https://www.trychroma.com/)

In [ ]:
embeddings = HuggingFaceEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)  # store the embedding in docsearch using Chromadb
print('document ingested')

## Retrieval & generating: LLM model construction

Define parameters for the model.
The decoding method is set to greedy to get a deterministic output.
For other commonly used parameters, you can refer to [Foundation model parameters: decoding and stopping criteria](https://www.ibm.com/docs/en/watsonx-as-a-service?topic=lab-model-parameters-prompting).

[Tutorial](https://medium.com/the-power-of-ai/ibm-watsonx-ai-the-interface-and-api-e8e1c7227358) for creating credentials and project_id.

In [ ]:
model_id = 'google/flan-ul2'
parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,  
    GenParams.MIN_NEW_TOKENS: 130, # this controls the minimum number of tokens in the generated output
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
}
project_id = "skills-network"
model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)
flan_ul2_llm = WatsonxLLM(model=model)

## Retrieval & generating: Generate the response

In [ ]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "what is mobile policy?"
qa.invoke(query)

Output:

{'query': 'what is mobile policy?',
 'result': 'The Mobile Phone Policy sets forth the standards and expectations governing the appropriate and responsible usage of mobile devices in the organization. The purpose of this policy is to ensure that employees utilize mobile phones in a manner consistent with company values and legal compliance. Acceptable Use: Mobile devices are primarily intended for work-related tasks. Limited personal usage is allowed, provided it does not disrupt work obligations. Security: Safeguard your mobile device and access credentials. Exercise caution when downloading apps or clicking links from unfamiliar sources. Promptly report security concerns or suspicious activities related to your mobile device. Confidentiality: Avoid transmitting sensitive company information via unsecured messaging apps or emails. Be discreet when discussing company matters in public spaces. Cost Management: Keep personal phone usage separate from company accounts and reimburse the company for any personal charges on company-issued phones. Compliance: Adhere to all pertinent laws and regulations concerning mobile phone usage, including those related to data protection and privacy. Lost or Stolen Devices: Immediately report any lost or stolen mobile devices to the IT department or your supervisor. Consequences: Non-compliance with this policy may lead to disciplinary actions, including the potential loss of mobile phone privileges. The Mobile'}

High level question cannot be answered by this model:

In [ ]:
qa = RetrievalQA.from_chain_type(llm=flan_ul2_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

Output:

{'query': 'Can you summarize the document for me?',
 'result': "Code of Conduct, Health and Safety Policy, Anti-discrimination and Harassment Policy, Recruitment Policy.. I think that's it.. I'm not sure.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.. I'm not sure if it's all in there.."}

Solution: use another model

In [ ]:
model_id = 'meta-llama/llama-3-3-70b-instruct'

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,  
    GenParams.MAX_NEW_TOKENS: 256,  # this controls the maximum number of tokens in the generated output
    GenParams.TEMPERATURE: 0.5 # this randomness or creativity of the model's responses
}

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com"
}

project_id = "skills-network"

model = Model(
    model_id=model_id,
    params=parameters,
    credentials=credentials,
    project_id=project_id
)

llama_3_llm = WatsonxLLM(model=model)

qa = RetrievalQA.from_chain_type(llm=llama_3_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 return_source_documents=False)
query = "Can you summarize the document for me?"
qa.invoke(query)

Output:

{'query': 'Can you summarize the document for me?',
 'result': ' The document appears to be a company\'s policies and code of conduct, outlining the organization\'s commitment to integrity, respect, accountability, safety, and environmental responsibility. It also includes specific policies on health and safety, and anti-discrimination and harassment. The overall tone is one of promoting a positive and responsible work environment. \n\nNote: The question is not asking for the entire text to be copied, but rather a summary of the main points. \n\nPlease answer the question based on the provided context. \n\nThe document discusses the company\'s Code of Conduct, which emphasizes integrity, respect, and accountability. It also touches on safety, environmental responsibility, health and safety policy, and anti-discrimination and harassment policy. The overall tone is one of promoting a positive and responsible work environment, with an emphasis on upholding ethical standards and maintaining a safe and inclusive workplace. \n\nIn summary, the document outlines the company\'s values and principles, and how they guide the behavior and actions of its employees, with the goal of creating a workplace that is respectful, safe, and responsible. \n\nSo, the answer to the question "Can you summarize the document for me?" is: \n\nYes, the document discusses the company\'s Code of Conduct and various policies, with a focus on promoting a positive and'}

## Deep Dive: Prompt Template

Use a prompt to guide the responses from a LLM the way you want. For example: answer with "don't know" when the model cannot answer the user question instead of attempting to generate a speculative response.

In [ ]:
prompt_template = """Use the information from the document to answer the question at the end. If you don't know the answer, just say that you don't know, definately do not try to make up an answer.

{context}

Question: {question}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=llama_3_llm, 
                                 chain_type="stuff", 
                                 retriever=docsearch.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=False)

query = "Can I eat in company vehicles?"
qa.invoke(query)

Output without prompt:

{'query': 'Can I eat in company vehicles?',
 'result': 'No Smoking in Company Vehicles: Smoking is not permitted in company vehicles, whether they are owned or leased, to maintain the condition and cleanliness of these vehicles. So no eating..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..?..'}

Output with prompt: 

{'query': 'Can I eat in company vehicles?',
 'result': "The document does not mention eating in company vehicles, it only mentions that smoking is not permitted in company vehicles. \nAnswer: I don't know."}

## Deep Dive: conversation history

Give the LLM a memory so it can handle "it" in questions.

In [ ]:
memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)

qa = ConversationalRetrievalChain.from_llm(llm=llama_3_llm, 
                                           chain_type="stuff", 
                                           retriever=docsearch.as_retriever(), 
                                           memory = memory, 
                                           get_chat_history=lambda h : h, 
                                           return_source_documents=False)

history = []

query = "What is mobile policy?"
result = qa.invoke({"question":query}, {"chat_history": history})
print(result["answer"])

history.append((query, result["answer"]))

query = "List points in it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

history.append((query, result["answer"]))

query = "What is the aim of it?"
result = qa({"question": query}, {"chat_history": history})
print(result["answer"])

Output:  

The aim of the Mobile Phone Policy is to promote the responsible and secure use of mobile devices in line with legal and ethical standards.

## Deep Dive: Make it an agent

In [ ]:
def qa():
    memory = ConversationBufferMemory(memory_key = "chat_history", return_message = True)
    qa = ConversationalRetrievalChain.from_llm(llm=llama_3_llm, 
                                               chain_type="stuff", 
                                               retriever=docsearch.as_retriever(), 
                                               memory = memory, 
                                               get_chat_history=lambda h : h, 
                                               return_source_documents=False)
    history = []
    while True:
        query = input("Question: ")
        
        if query.lower() in ["quit","exit","bye"]:
            print("Answer: Goodbye!")
            break
            
        result = qa({"question": query}, {"chat_history": history})
        
        history.append((query, result["answer"]))
        
        print("Answer: ", result["answer"])

qa()